# OXCART Philatelic RAG System

Sistema completo de indexación y búsqueda semántica para documentos filatélicos.

**Funcionalidades:**
- 📄 Indexación automática de todos los JSONs philatelic
- 🔍 Búsqueda semántica avanzada con filtros filatélicos
- 🤖 RAG básico con LLM para responder preguntas
- 📊 Dashboard de estadísticas y validación
- 🌐 Interfaz Gradio para consultas interactivas

**Requisitos:**
- Weaviate corriendo en Docker: `docker-compose up -d`
- OpenAI API key configurada en `.env`
- JSONs philatelic en `results/final_jsons/`

## 1. Setup y Configuración

Configuración inicial del entorno y carga de librerías.

In [24]:
import os
import json
import glob
import time
from pathlib import Path
from typing import Dict, Any, List, Optional
from datetime import datetime

# Cargar variables de entorno
from dotenv import load_dotenv
load_dotenv()

# Imports de terceros
import pandas as pd

print("✅ Imports básicos completados")

✅ Imports básicos completados


In [25]:
# Verificar variables de entorno
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
WEAVIATE_URL = os.getenv('WEAVIATE_URL', 'http://localhost:8083')
PHILATELIC_JSONS_DIR = os.getenv('PHILATELIC_JSONS_DIR', './results/final_jsons')
COLLECTION_NAME = os.getenv('WEAVIATE_COLLECTION_NAME', 'Oxcart')

print(f"🔧 Configuración:")
print(f"   • Weaviate URL: {WEAVIATE_URL}")
print(f"   • JSONs Directory: {PHILATELIC_JSONS_DIR}")
print(f"   • Collection Name: {COLLECTION_NAME}")
print(f"   • OpenAI API Key: {'✅ Configurada' if OPENAI_API_KEY else '❌ Falta configurar'}")

if not OPENAI_API_KEY:
    print("\\n⚠️  IMPORTANTE: Configura tu OPENAI_API_KEY en el archivo .env")
    print("   Copia .env.example a .env y agrega tu API key")

# Verificar que el directorio de JSONs existe
if not os.path.exists(PHILATELIC_JSONS_DIR):
    print(f"\\n⚠️  Directorio {PHILATELIC_JSONS_DIR} no encontrado")
    print("   Asegúrate de haber procesado documentos con el Dolphin parser")
else:
    json_files = glob.glob(os.path.join(PHILATELIC_JSONS_DIR, '*_final.json'))
    print(f"\\n📁 Encontrados {len(json_files)} archivos JSON filatélicos")
    if json_files:
        print("   Ejemplos:")
        for file in json_files[:3]:
            print(f"   • {os.path.basename(file)}")
        if len(json_files) > 3:
            print(f"   • ... y {len(json_files) - 3} más")

🔧 Configuración:
   • Weaviate URL: http://localhost:8083
   • JSONs Directory: ./results/final_jsons
   • Collection Name: Oxcart
   • OpenAI API Key: ✅ Configurada
\n📁 Encontrados 1424 archivos JSON filatélicos
   Ejemplos:
   • 1901 National Theater of Costa Rica Yankowski_final.json
   • 1947_Overprint_final.json
   • 1967 Postal and social Plan_Luis Escalante_final.json
   • ... y 1421 más


In [26]:
# Recargar módulos del sistema OXCART para obtener las últimas mejoras
import importlib

# Recargar el módulo philatelic_weaviate para obtener las funciones actualizadas
try:
    import philatelic_weaviate
    importlib.reload(philatelic_weaviate)
    print("🔄 Módulo philatelic_weaviate recargado")
except ImportError:
    pass

from philatelic_weaviate import (
    create_weaviate_client,
    create_oxcart_collection,
    index_philatelic_document,
    search_chunks_semantic,
    get_collection_stats,
    transform_chunk_to_weaviate
)

from philatelic_chunk_schema import (
    PhilatelicDocument,
    PhilatelicChunk,
    validate_chunk_structure,
    get_chunk_summary
)

print("✅ Módulos OXCART cargados exitosamente con las mejoras más recientes")

Philatelic Weaviate Integration v2.1 cargado exitosamente
Funciones disponibles:
   - create_weaviate_client()
   - create_oxcart_collection()
   - index_philatelic_document()
   - search_chunks_semantic()
   - get_collection_stats()
🔄 Módulo philatelic_weaviate recargado
✅ Módulos OXCART cargados exitosamente con las mejoras más recientes


## 2. Descubrimiento de Documentos

Escanear automáticamente todos los archivos JSON philatelic disponibles.

In [27]:
def discover_philatelic_jsons(directory: str) -> List[Dict[str, Any]]:
    """
    Descubrir todos los archivos JSON philatelic en el directorio.
    Cuenta chunks ya indexados vs pendientes vs truncados vs no truncados.
    
    Returns:
        Lista de diccionarios con información de cada archivo
    """
    # Importar tqdm para progress bar
    from tqdm import tqdm
    
    json_files = []
    
    # Buscar archivos *_final.json
    pattern = os.path.join(directory, "*_final.json")
    philatelic_files = glob.glob(pattern)
    
    print(f"🔍 Buscando archivos en: {directory}")
    print(f"📋 Patrón de búsqueda: *_final.json")
    print(f"📄 Archivos encontrados: {len(philatelic_files)}")
    
    # Progress bar para descubrimiento
    for file_path in tqdm(philatelic_files, desc="📄 Analizando documentos", unit="doc"):
        try:
            # Obtener información del archivo
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
            file_name = os.path.basename(file_path)
            doc_id = file_name.replace("_final.json", "")
            
            # Cargar archivo para obtener estadísticas básicas
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            chunks = data.get("chunks", [])
            page_count = data.get("page_count", len(chunks))  # Estimado si no está disponible
            
            # Calcular estadísticas básicas y contar chunks por estado
            total_text_length = 0
            chunk_types = {}
            chunks_indexed = 0
            chunks_pending = 0
            chunks_truncated = 0
            chunks_not_truncated = 0
            chunks_truncated_unknown = 0
            
            for chunk in chunks:
                chunk_text = chunk.get("text", "") or chunk.get("content", "")
                total_text_length += len(chunk_text)
                
                chunk_type = chunk.get("chunk_type", "text")
                chunk_types[chunk_type] = chunk_types.get(chunk_type, 0) + 1
                
                # Verificar estado de indexación
                if chunk.get("indexed", False):
                    chunks_indexed += 1
                else:
                    chunks_pending += 1
                
                # Verificar estado de truncado (trazabilidad completa)
                truncated_flag = chunk.get("truncated", None)
                if truncated_flag is True:
                    chunks_truncated += 1
                elif truncated_flag is False:
                    chunks_not_truncated += 1
                else:
                    # Chunk sin marcar (sin procesar aún)
                    chunks_truncated_unknown += 1
            
            avg_chunk_length = total_text_length / len(chunks) if chunks else 0
            
            json_files.append({
                "file_path": file_path,
                "file_name": file_name,
                "doc_id": doc_id,
                "file_size_mb": round(file_size, 2),
                "chunks_count": len(chunks),
                "chunks_indexed": chunks_indexed,
                "chunks_pending": chunks_pending,
                "chunks_truncated": chunks_truncated,
                "chunks_not_truncated": chunks_not_truncated,
                "chunks_truncated_unknown": chunks_truncated_unknown,
                "page_count": page_count,
                "total_text_length": total_text_length,
                "avg_chunk_length": round(avg_chunk_length, 1),
                "chunk_types": chunk_types,
                "data": data  # Guardar datos para indexación
            })
            
        except Exception as e:
            print(f"   ❌ Error procesando {file_path}: {e}")
    
    # Mostrar resumen de indexación y trazabilidad
    if json_files:
        total_chunks = sum(f["chunks_count"] for f in json_files)
        total_indexed = sum(f["chunks_indexed"] for f in json_files)
        total_pending = sum(f["chunks_pending"] for f in json_files)
        total_truncated = sum(f["chunks_truncated"] for f in json_files)
        total_not_truncated = sum(f["chunks_not_truncated"] for f in json_files)
        total_truncated_unknown = sum(f["chunks_truncated_unknown"] for f in json_files)
        
        print(f"\n📊 ESTADO DE INDEXACIÓN:")
        print(f"   📦 Total chunks: {total_chunks:,}")
        print(f"   ✅ Ya indexados: {total_indexed:,} ({(total_indexed/total_chunks)*100:.1f}%)")
        print(f"   ⏳ Pendientes: {total_pending:,} ({(total_pending/total_chunks)*100:.1f}%)")
        
        print(f"\n🔍 TRAZABILIDAD DE TRUNCADO:")
        print(f"   ✂️ Truncados: {total_truncated:,} ({(total_truncated/total_chunks)*100:.1f}%)")
        print(f"   📏 No truncados: {total_not_truncated:,} ({(total_not_truncated/total_chunks)*100:.1f}%)")
        print(f"   ❓ Sin procesar: {total_truncated_unknown:,} ({(total_truncated_unknown/total_chunks)*100:.1f}%)")
        
        if total_pending == 0:
            print(f"   🎉 ¡Todos los chunks están indexados!")
        elif total_indexed > 0:
            print(f"   🔄 Se continuará desde donde se quedó")
            
        # Información sobre trazabilidad
        if total_truncated > 0 or total_not_truncated > 0:
            total_processed = total_truncated + total_not_truncated
            print(f"   💡 {total_processed:,} chunks con trazabilidad completa de truncado")
            if total_truncated > 0:
                print(f"   📄 Los chunks truncados mantienen texto original en 'text_original'")
    
    return json_files

print("✅ Función de descubrimiento mejorada con trazabilidad completa de truncado")

✅ Función de descubrimiento mejorada con trazabilidad completa de truncado


In [28]:
# Descubrir archivos
discovered_files = discover_philatelic_jsons(PHILATELIC_JSONS_DIR)

print(f"\\n📊 RESUMEN DE DESCUBRIMIENTO:")
print(f"   📄 Archivos encontrados: {len(discovered_files)}")

if discovered_files:
    # === ESTADÍSTICAS BÁSICAS ===
    total_chunks = sum(f["chunks_count"] for f in discovered_files)
    total_indexed = sum(f["chunks_indexed"] for f in discovered_files)
    total_pending = sum(f["chunks_pending"] for f in discovered_files)
    total_pages = sum(f["page_count"] for f in discovered_files)
    total_size = sum(f["file_size_mb"] for f in discovered_files)
    total_text_length = sum(f["total_text_length"] for f in discovered_files)
    
    print(f"   📦 Total chunks: {total_chunks:,}")
    print(f"   ✅ Ya indexados: {total_indexed:,} ({(total_indexed/total_chunks)*100:.1f}%)")
    print(f"   ⏳ Pendientes: {total_pending:,} ({(total_pending/total_chunks)*100:.1f}%)")
    print(f"   📄 Total páginas: {total_pages:,}")
    print(f"   💾 Tamaño total: {total_size:.1f} MB")
    
    # === ESTADÍSTICAS AVANZADAS DE CHUNKS ===
    if total_chunks > 0:
        # Promedio global de tamaño de chunks
        avg_chunk_size_global = total_text_length / total_chunks
        avg_chunks_per_doc = total_chunks / len(discovered_files)
        
        # Distribución de tipos de chunks
        all_chunk_types = {}
        chunk_sizes = []
        
        for f in discovered_files:
            chunk_sizes.extend([f["avg_chunk_length"]] * f["chunks_count"])
            for chunk_type, count in f["chunk_types"].items():
                all_chunk_types[chunk_type] = all_chunk_types.get(chunk_type, 0) + count
        
        # Estadísticas de tamaño
        min_chunk_size = min(chunk_sizes) if chunk_sizes else 0
        max_chunk_size = max(chunk_sizes) if chunk_sizes else 0
        
        print(f"\\n📊 ESTADÍSTICAS DE CHUNKS:")
        print(f"   📏 Tamaño promedio global: {avg_chunk_size_global:.0f} caracteres")
        print(f"   📈 Promedio chunks/documento: {avg_chunks_per_doc:.1f}")
        print(f"   📉 Rango de tamaños: {min_chunk_size:.0f} - {max_chunk_size:.0f} chars")
        
        # Top tipos de chunks
        print(f"   🏷️ Tipos más comunes:")
        sorted_types = sorted(all_chunk_types.items(), key=lambda x: x[1], reverse=True)
        for chunk_type, count in sorted_types[:5]:
            percentage = (count / total_chunks) * 100
            print(f"      • {chunk_type}: {count:,} ({percentage:.1f}%)")
    
    # === ESTIMACIÓN DE COSTOS OPENAI ===
    if total_pending > 0:
        print(f"\\n💰 ESTIMACIÓN DE COSTOS OPENAI (SOLO CHUNKS PENDIENTES):")
        
        # Configuración del modelo de embeddings
        # text-embedding-3-large: $0.00013 per 1K tokens (más reciente y eficiente)
        EMBEDDING_MODEL = "text-embedding-3-large"
        COST_PER_1K_TOKENS = 0.00013  # USD
        CHARS_PER_TOKEN = 4  # Aproximación para texto en español
        
        # Calcular tokens estimados solo para chunks pendientes
        pending_text_length = 0
        for f in discovered_files:
            if f["chunks_pending"] > 0:
                # Estimar texto de chunks pendientes (proporcionalmente)
                pending_ratio = f["chunks_pending"] / f["chunks_count"]
                pending_text_length += f["total_text_length"] * pending_ratio
        
        estimated_tokens = pending_text_length / CHARS_PER_TOKEN
        estimated_cost = (estimated_tokens / 1000) * COST_PER_1K_TOKENS
        
        print(f"   🤖 Modelo: {EMBEDDING_MODEL}")
        print(f"   📝 Caracteres pendientes: {pending_text_length:,.0f}")
        print(f"   🎯 Tokens estimados: {estimated_tokens:,.0f}")
        print(f"   💵 Costo estimado: ${estimated_cost:.4f} USD")
        
        # Estimaciones adicionales útiles
        if estimated_cost > 0:
            cost_per_chunk = estimated_cost / total_pending
            docs_with_pending = sum(1 for f in discovered_files if f["chunks_pending"] > 0)
            cost_per_document = estimated_cost / docs_with_pending if docs_with_pending > 0 else 0
            
            print(f"   📊 Costo por chunk pendiente: ${cost_per_chunk:.6f} USD")
            print(f"   📄 Costo por documento con pendientes: ${cost_per_document:.4f} USD")
            
            # Rangos de referencia
            if estimated_cost < 0.01:
                cost_range = "💚 Muy bajo"
            elif estimated_cost < 0.10:
                cost_range = "💙 Bajo"
            elif estimated_cost < 1.00:
                cost_range = "💛 Moderado"
            elif estimated_cost < 5.00:
                cost_range = "🧡 Alto"
            else:
                cost_range = "❤️ Muy alto"
            
            print(f"   📈 Rango de costo: {cost_range}")
    else:
        print(f"\\n🎉 ¡No hay chunks pendientes para indexar!")
        print(f"   💰 Costo estimado: $0.00 USD")
    
    # === ADVERTENCIAS Y NOTAS ===
    print(f"\\n⚠️ NOTAS IMPORTANTES:")
    print(f"   • Solo se procesarán chunks pendientes (sin flag 'indexed': true)")
    print(f"   • Los chunks exitosos se marcarán automáticamente como indexados")
    print(f"   • Los archivos JSON se actualizarán automáticamente")
    print(f"   • Las futuras ejecuciones continuarán donde se quedó")
    print(f"   • Los costos son estimaciones basadas en {CHARS_PER_TOKEN} chars/token")
    
else:
    print(f"   ⚠️ No se encontraron archivos *_final.json en {PHILATELIC_JSONS_DIR}")
    print(f"   💡 Asegúrate de haber procesado documentos con el Dolphin parser")

🔍 Buscando archivos en: ./results/final_jsons
📋 Patrón de búsqueda: *_final.json
📄 Archivos encontrados: 1424


📄 Analizando documentos: 100%|██████████| 1424/1424 [00:10<00:00, 132.48doc/s]



📊 ESTADO DE INDEXACIÓN:
   📦 Total chunks: 192,680
   ✅ Ya indexados: 192,644 (100.0%)
   ⏳ Pendientes: 36 (0.0%)

🔍 TRAZABILIDAD DE TRUNCADO:
   ✂️ Truncados: 7,150 (3.7%)
   📏 No truncados: 184,960 (96.0%)
   ❓ Sin procesar: 570 (0.3%)
   🔄 Se continuará desde donde se quedó
   💡 192,110 chunks con trazabilidad completa de truncado
   📄 Los chunks truncados mantienen texto original en 'text_original'
\n📊 RESUMEN DE DESCUBRIMIENTO:
   📄 Archivos encontrados: 1424
   📦 Total chunks: 192,680
   ✅ Ya indexados: 192,644 (100.0%)
   ⏳ Pendientes: 36 (0.0%)
   📄 Total páginas: 22,940
   💾 Tamaño total: 1129.7 MB
\n📊 ESTADÍSTICAS DE CHUNKS:
   📏 Tamaño promedio global: 1804 caracteres
   📈 Promedio chunks/documento: 135.3
   📉 Rango de tamaños: 47 - 6620 chars
   🏷️ Tipos más comunes:
      • text: 148,562 (77.1%)
      • header: 22,745 (11.8%)
      • marginalia: 15,052 (7.8%)
      • decree: 2,794 (1.5%)
      • footer: 1,742 (0.9%)
\n💰 ESTIMACIÓN DE COSTOS OPENAI (SOLO CHUNKS PENDIENTES)

In [29]:
# Mostrar tabla resumen de archivos
if discovered_files:
    files_df = pd.DataFrame([
        {
            "Documento": f["doc_id"],
            "Chunks": f["chunks_count"],
            "Páginas": f["page_count"],
            "Tamaño (MB)": f["file_size_mb"],
            "Promedio chunk": f["avg_chunk_length"],
            "Tipos principales": ", ".join([f"{k}: {v}" for k, v in list(f["chunk_types"].items())[:3]])
        }
        for f in discovered_files
    ])
    
    print("\n📋 DOCUMENTOS ENCONTRADOS:")
    print(files_df.to_string(index=False))
else:
    print("\n❌ No hay documentos para mostrar")


📋 DOCUMENTOS ENCONTRADOS:
                                                                 Documento  Chunks  Páginas  Tamaño (MB)  Promedio chunk                            Tipos principales
                             1901 National Theater of Costa Rica Yankowski      64       16         0.18           564.1              text: 38, header: 21, footer: 2
                                                            1947_Overprint      66       16         0.54          2291.0        marginalia: 10, header: 15, decree: 2
                                1967 Postal and social Plan_Luis Escalante      76       16         0.22           770.8              header: 15, text: 51, decree: 1
                                           1983 Issue Cesar Sancho Frame 1      51       16         0.44          2139.6          marginalia: 8, text: 32, header: 11
                                           1983 Issue Cesar Sancho Frame 2      42       16         0.20          1043.1          marginalia: 4

## 3. Configuración de Weaviate

Conectar a Weaviate y crear la colección con esquema optimizado.

In [30]:
import weaviate

weaviate.__version__

'4.16.9'

In [31]:
# Conectar a Weaviate
print("🔌 Conectando a Weaviate...")

try:
    client = create_weaviate_client(WEAVIATE_URL, OPENAI_API_KEY)
    print("✅ Conexión exitosa")
    
    # Verificar que Weaviate esté funcionando
    meta = client.get_meta()
    print(f"📊 Weaviate versión: {meta.get('version', 'unknown')}")
    
    # Verificar si la colección existe
    try:
        collections = client.collections.list_all()
        collection_names = [col.name for col in collections]
        
        if COLLECTION_NAME in collection_names:
            collection = client.collections.get(COLLECTION_NAME)
            total_objects = collection.aggregate.over_all(total_count=True).total_count
            print(f"📊 Colección '{COLLECTION_NAME}' existe con {total_objects} documentos")
        else:
            print(f"📝 Colección '{COLLECTION_NAME}' no existe (se creará durante la indexación)")
    except Exception as e:
        print(f"⚠️ No se pudo verificar colecciones: {e}")
        
except Exception as e:
    print(f"❌ Error conectando a Weaviate: {e}")
    print("💡 Asegúrate de que Weaviate esté corriendo:")
    print("   docker-compose up -d")
    client = None

🔌 Conectando a Weaviate...
Conectado a Weaviate en http://localhost:8083
✅ Conexión exitosa
📊 Weaviate versión: 1.32.4
⚠️ No se pudo verificar colecciones: 'str' object has no attribute 'name'


c:\Users\VM-SERVER\Desktop\Oxcart RAG\.venv-clean\lib\site-packages\weaviate\warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
C:\Users\VM-SERVER\AppData\Local\Temp\ipykernel_107544\3941462923.py:5: ResourceWarning: unclosed <socket.socket fd=3048, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('::1', 45615, 0, 0), raddr=('::1', 8083, 0, 0)>
  client = create_weaviate_client(WEAVIATE_URL, OPENAI_API_KEY)


In [32]:
#client.collections.delete(COLLECTION_NAME)

In [33]:
# Crear colección Oxcart
if client:
    print("\n🏗️ Configurando colección Oxcart...")
    
    collection_created = create_oxcart_collection(client, COLLECTION_NAME)
    
    if collection_created:
        print("✅ Colección lista para indexación")
        
        # Mostrar estadísticas de la colección
        stats = get_collection_stats(client, COLLECTION_NAME)
        if stats:
            print(f"📊 Chunks actuales en Weaviate: {stats.get('total_chunks', 0)}")
            if stats.get('documents'):
                print(f"📄 Documentos indexados: {list(stats['documents'].keys())}")
    else:
        print("❌ Error configurando colección")
        client = None
else:
    print("⚠️ Saltando configuración de colección (sin conexión)")


🏗️ Configurando colección Oxcart...
ADVERTENCIA: Coleccion 'Oxcart' ya existe
INFORMACION: Usando coleccion existente
✅ Colección lista para indexación
📊 Chunks actuales en Weaviate: 193144
📄 Documentos indexados: ['Mena 2014', 'Mena 2018 CRPC ', 'Scott 2024', 'The Postal History Frajola Mayer', 'Catálogo Scott CR 2024', 'Forgeries Articles Raul Hernandez', 'Nordberg Collection Gold', 'Costa Rica 1881-82 1883 UPU Surcharge Forgeries Fantasies', 'Pinto Collection', 'OXCART151', 'CRF 152', 'Costa Rica Coat of Arms 1892 Luis Fernando Diaz', 'Mayer Costa Rica', 'CRF 75-76', 'CRF 56', 'Index2020 Oxcart Subject Final', 'CRF 49-50', 'OXCART100', 'CRF 44-45', 'filatelista 12 b', 'filatelista 12 Café Establecimiento de una República', 'filatelista 12 a', 'CRF 65-66', 'CRF 144', 'CRF 151', 'CRF 155', 'CRF 77-82', 'CRF 67-68', 'CRF 85', 'Escalante Collection', 'CRF 69-70', 'Cat First Flights Wayman 2020', 'CRF 10', 'CRF 43', 'CRF 83-84', 'CRF 150', 'CRF 31-34', 'CRF 88', 'OXCART144', 'CRF 64', '

## 4. Indexación Automática

Indexar automáticamente todos los documentos philatelic en Weaviate.

In [34]:
def index_all_documents(client, discovered_files: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Indexar todos los documentos descubiertos en Weaviate con progress bars, persistencia
    y manejo inteligente de chunks largos.
    
    Returns:
        Dict con resultados de indexación
    """
    from tqdm import tqdm
    import json
    
    if not client:
        return {"error": "No hay conexión a Weaviate"}
    
    if not discovered_files:
        return {"error": "No hay documentos para indexar"}
    
    # Filtrar documentos con chunks pendientes
    documents_to_process = []
    total_pending_chunks = 0
    
    for file_info in discovered_files:
        chunks_pending = file_info.get("chunks_pending", 0)
        if chunks_pending > 0:
            documents_to_process.append(file_info)
            total_pending_chunks += chunks_pending
    
    if not documents_to_process:
        print("🎉 ¡Todos los documentos ya están completamente indexados!")
        return {
            "total_documents": len(discovered_files),
            "successful_documents": len(discovered_files),
            "failed_documents": 0,
            "total_chunks_indexed": 0,
            "total_chunks_pending": 0,
            "all_indexed": True
        }
    
    print(f"🚀 INICIANDO INDEXACIÓN MASIVA ROBUSTA CON MANEJO DE CHUNKS LARGOS")
    print(f"📄 Documentos con chunks pendientes: {len(documents_to_process)}")
    print(f"📦 Total chunks pendientes: {total_pending_chunks:,}")
    print(f"✂️ Chunks largos serán truncados automáticamente a 12,000 caracteres")
    print("=" * 60)
    
    indexing_results = []
    total_chunks_indexed = 0
    total_chunks_failed = 0
    total_chunks_truncated = 0
    total_chars_saved = 0
    documents_updated = 0
    
    start_time = time.time()
    
    # Progress bar principal para documentos
    doc_pbar = tqdm(
        documents_to_process, 
        desc="📄 Procesando documentos", 
        unit="doc",
        position=0
    )
    
    # Progress bar secundaria para chunks del documento actual
    chunk_pbar = None
    
    for i, file_info in enumerate(doc_pbar):
        doc_id = file_info["doc_id"]
        document = file_info["data"]
        chunks_count = file_info["chunks_count"]
        chunks_pending = file_info["chunks_pending"]
        file_path = file_info["file_path"]
        
        doc_pbar.set_description(f"📄 Procesando {doc_id}")
        
        # Progress bar para chunks de este documento
        if chunks_pending > 0:
            chunk_pbar = tqdm(
                total=chunks_pending,
                desc=f"  📦 Indexando chunks",
                unit="chunk",
                position=1,
                leave=False
            )
            
            # Callback para actualizar progress bar de chunks
            def update_chunk_progress(successful_count):
                if chunk_pbar:
                    chunk_pbar.update(successful_count)
        else:
            update_chunk_progress = None
        
        try:
            # Indexar documento usando la función mejorada
            result = index_philatelic_document(
                client, 
                document, 
                COLLECTION_NAME,
                progress_callback=update_chunk_progress
            )
            
            # Guardar resultado
            chunks_indexed = result.get("successful", 0)
            chunks_failed = len(result.get("errors", []))
            chunks_marked = result.get("chunks_marked_as_indexed", 0)
            
            # Estadísticas de validación (chunks truncados)
            validation_stats = result.get("validation_stats", {})
            chunks_truncated_doc = validation_stats.get("truncated_chunks", 0)
            chars_saved_doc = validation_stats.get("total_chars_saved", 0)
            
            indexing_results.append({
                "doc_id": doc_id,
                "success": chunks_indexed > 0 or result.get("already_indexed", False),
                "chunks_indexed": chunks_indexed,
                "chunks_failed": chunks_failed,
                "chunks_marked": chunks_marked,
                "chunks_truncated": chunks_truncated_doc,
                "chars_saved": chars_saved_doc,
                "already_indexed": result.get("already_indexed", False),
                "errors": result.get("errors", []),
                "validation_stats": validation_stats
            })
            
            total_chunks_indexed += chunks_indexed
            total_chunks_failed += chunks_failed
            total_chunks_truncated += chunks_truncated_doc
            total_chars_saved += chars_saved_doc
            
            # Actualizar descripción del progress bar
            status_parts = []
            if chunks_marked > 0:
                status_parts.append(f"{chunks_marked} marcados")
            if chunks_truncated_doc > 0:
                status_parts.append(f"{chunks_truncated_doc} truncados")
            
            if status_parts:
                doc_pbar.set_postfix_str(f"✅ {', '.join(status_parts)}")
            
            # Guardar archivo JSON actualizado si hay chunks marcados
            if chunks_marked > 0:
                try:
                    with open(file_path, 'w', encoding='utf-8') as f:
                        json.dump(document, f, ensure_ascii=False, indent=2)
                    documents_updated += 1
                except Exception as save_error:
                    print(f"⚠️ Error guardando {file_path}: {save_error}")
            elif result.get("already_indexed"):
                doc_pbar.set_postfix_str("✅ Ya indexado")
            
        except Exception as e:
            print(f"❌ Error indexando {doc_id}: {e}")
            indexing_results.append({
                "doc_id": doc_id,
                "success": False,
                "error": str(e)
            })
        
        finally:
            # Cerrar progress bar de chunks
            if chunk_pbar:
                chunk_pbar.close()
    
    doc_pbar.close()
    total_time = time.time() - start_time
    
    # Resumen final
    successful_docs = sum(1 for r in indexing_results if r.get("success", False))
    
    summary = {
        "total_documents": len(discovered_files),
        "documents_processed": len(documents_to_process),
        "successful_documents": successful_docs,
        "failed_documents": len(documents_to_process) - successful_docs,
        "documents_updated": documents_updated,
        "total_chunks_indexed": total_chunks_indexed,
        "total_chunks_failed": total_chunks_failed,
        "total_chunks_truncated": total_chunks_truncated,
        "total_chars_saved": total_chars_saved,
        "total_time_seconds": total_time,
        "avg_time_per_document": total_time / len(documents_to_process) if documents_to_process else 0,
        "chunks_per_second": total_chunks_indexed / total_time if total_time > 0 else 0,
        "results": indexing_results
    }
    
    print("\\n" + "=" * 60)
    print("📊 RESUMEN FINAL DE INDEXACIÓN:")
    print(f"   ✅ Documentos procesados: {len(documents_to_process)}")
    print(f"   ✅ Documentos exitosos: {successful_docs}")
    print(f"   💾 Archivos JSON actualizados: {documents_updated}")
    print(f"   📦 Chunks indexados: {total_chunks_indexed:,}")
    print(f"   ❌ Chunks fallidos: {total_chunks_failed:,}")
    
    # Mostrar estadísticas de truncado si hay chunks truncados
    if total_chunks_truncated > 0:
        print(f"   ✂️ Chunks truncados exitosamente: {total_chunks_truncated:,}")
        print(f"   💾 Caracteres removidos: {total_chars_saved:,}")
        truncation_rate = (total_chunks_truncated / (total_chunks_indexed + total_chunks_failed)) * 100
        print(f"   📊 Tasa de truncado: {truncation_rate:.1f}%")
        print("   💡 Los chunks truncados mantienen información clave del inicio")
    
    print(f"   ⏱️ Tiempo total: {total_time:.1f} segundos")
    print(f"   🚀 Velocidad: {summary['chunks_per_second']:.1f} chunks/segundo")
    
    success_rate = (total_chunks_indexed / (total_chunks_indexed + total_chunks_failed)) * 100 if (total_chunks_indexed + total_chunks_failed) > 0 else 0
    print(f"   📈 Tasa de éxito: {success_rate:.1f}%")
    
    return summary

print("✅ Función de indexación mejorada con manejo inteligente de chunks largos")

✅ Función de indexación mejorada con manejo inteligente de chunks largos


In [35]:
# === PRUEBA CON MANEJO DE CHUNKS LARGOS ===
# Cambiar test_mode = True para probar la solución de chunks largos
test_mode = False  # ✅ ACTIVADO para probar solución de chunks largos
if test_mode and discovered_files:
    print("🧪 MODO PRUEBA: Probando manejo inteligente de chunks largos")
    print("✂️ Esta prueba validará que chunks > 30,000 caracteres sean truncados automáticamente")
    test_files = [discovered_files[2]]  # Solo el primer documento
    indexing_summary = index_all_documents(client, test_files)
    
    print("\\n🧪 ANÁLISIS DE PRUEBA:")
    if indexing_summary and "results" in indexing_summary:
        result = indexing_summary["results"][0]
        if result.get("chunks_truncated", 0) > 0:
            print(f"✅ ÉXITO: {result['chunks_truncated']} chunks fueron truncados automáticamente")
            print(f"💾 Caracteres removidos: {result['chars_saved']:,}")
            print("✅ No debería haber errores de 'maximum context length'")
        else:
            print("ℹ️ No se encontraron chunks que requirieran truncado en este documento")
        
        if result.get("chunks_indexed", 0) > 0:
            print(f"✅ ÉXITO: {result['chunks_indexed']} chunks indexados exitosamente")
        else:
            print("⚠️ No se indexó ningún chunk - revisar errores")
            
    print("\\n🧪 Prueba completada. Si no hay errores de 'maximum context length', la solución funciona.")
elif test_mode and not discovered_files:
    print("⚠️ No hay documentos para probar")
else:
    print("ℹ️ Modo prueba desactivado (test_mode = False)")

ℹ️ Modo prueba desactivado (test_mode = False)


# === INDEXACIÓN COMPLETA ===

In [36]:
# Ejecutar indexación
if client and discovered_files:
    print("🎯 ¿Proceder con la indexación robusta?")
    
    # Calcular solo chunks pendientes
    total_pending = sum(f.get("chunks_pending", 0) for f in discovered_files)
    docs_with_pending = sum(1 for f in discovered_files if f.get("chunks_pending", 0) > 0)
    
    if total_pending == 0:
        print("🎉 ¡Todos los chunks ya están indexados!")
        print("   No hay nada que procesar.")
        indexing_summary = {
            "all_indexed": True,
            "message": "Todos los chunks ya están indexados"
        }
    else:
        print(f"   📄 Documentos con chunks pendientes: {docs_with_pending}")
        print(f"   📦 Total chunks pendientes: {total_pending:,}")
        
        # Estimar tiempo mejorado (con rate limiting y reintentos)
        estimated_minutes = total_pending / 75  # Más conservador: ~75 chunks por minuto
        print(f"   ⏱️ Tiempo estimado: {estimated_minutes:.1f} minutos")
        print(f"   🔄 Incluye reintentos automáticos y manejo de rate limits")
        print(f"   💾 Los archivos JSON se actualizarán automáticamente")
        
        # Ejecutar indexación robusta
        indexing_summary = index_all_documents(client, discovered_files)
        
        # Guardar resultados
        results_file = "indexing_results_robust.json"
        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump(indexing_summary, f, ensure_ascii=False, indent=2)
        print(f"\\n💾 Resultados guardados en: {results_file}")
        
        # Mostrar resumen de archivos actualizados
        if indexing_summary.get("documents_updated", 0) > 0:
            print(f"\\n📝 ARCHIVOS JSON ACTUALIZADOS:")
            print(f"   • {indexing_summary['documents_updated']} archivos con chunks marcados como indexados")
            print(f"   • Las futuras ejecuciones saltarán automáticamente estos chunks")
    
else:
    print("⚠️ No se puede proceder con la indexación:")
    if not client:
        print("   - Sin conexión a Weaviate")
    if not discovered_files:
        print("   - No hay documentos para indexar")
    indexing_summary = None

🎯 ¿Proceder con la indexación robusta?
   📄 Documentos con chunks pendientes: 1
   📦 Total chunks pendientes: 36
   ⏱️ Tiempo estimado: 0.5 minutos
   🔄 Incluye reintentos automáticos y manejo de rate limits
   💾 Los archivos JSON se actualizarán automáticamente
🚀 INICIANDO INDEXACIÓN MASIVA ROBUSTA CON MANEJO DE CHUNKS LARGOS
📄 Documentos con chunks pendientes: 1
📦 Total chunks pendientes: 36
✂️ Chunks largos serán truncados automáticamente a 12,000 caracteres


📄 Procesando AFCR Boletín 51 julio 2013:   0%|          | 0/1 [00:00<?, ?doc/s]

📄 Indexando documento: AFCR Boletín 51 julio 2013
   📊 Total chunks: 54
   ✅ Ya indexados: 18
   ⏳ Pendientes: 36
   📄 Páginas: 9
🔍 VALIDACIÓN PREVIA DE CHUNKS
🔍 Validando 36 chunks para documento AFCR Boletín 51 julio 2013
   📏 Límite: 12,000 caracteres (3000 tokens)
   ✅ Chunks válidos: 36
   📊 Tasa de éxito: 100.0%
🔍 VALIDACIÓN PREVIA DE CHUNKS
🔍 Validando 36 chunks para documento AFCR Boletín 51 julio 2013
   📏 Límite: 12,000 caracteres (3000 tokens)
   ✅ Chunks válidos: 36
   📊 Tasa de éxito: 100.0%

🚀 INICIANDO INDEXACIÓN ROBUSTA
   📄 Documento: AFCR Boletín 51 julio 2013
   📦 Chunks originales: 36
   ✅ Chunks válidos: 36
   📦 Lotes de 50 chunks con máximo 3 reintentos


📄 Procesando AFCR Boletín 51 julio 2013: 100%|██████████| 1/1 [00:02<00:00,  2.74s/doc, ✅ 36 marcados]

   📦 Lote 1/1: 36/36 exitosos
   📊 Resumen final: 36/36 chunks válidos indexados (100.0%)
✅ Documento AFCR Boletín 51 julio 2013 indexado exitosamente
   📝 Chunks marcados como indexados: 36
\n============================================================
📊 RESUMEN FINAL DE INDEXACIÓN:
   ✅ Documentos procesados: 1
   ✅ Documentos exitosos: 1
   💾 Archivos JSON actualizados: 1
   📦 Chunks indexados: 36
   ❌ Chunks fallidos: 0
   ⏱️ Tiempo total: 2.7 segundos
   🚀 Velocidad: 13.1 chunks/segundo
   📈 Tasa de éxito: 100.0%
\n💾 Resultados guardados en: indexing_results_robust.json
\n📝 ARCHIVOS JSON ACTUALIZADOS:
   • 1 archivos con chunks marcados como indexados
   • Las futuras ejecuciones saltarán automáticamente estos chunks


## 5. Validación y Estadísticas

Verificar que la indexación fue exitosa y mostrar estadísticas detalladas.

In [37]:
# Validar indexación
if client:
    print("🔍 VALIDANDO INDEXACIÓN...")
    
    # Obtener estadísticas actuales
    current_stats = get_collection_stats(client, COLLECTION_NAME)
    
    if current_stats:
        print(f"\n📊 ESTADÍSTICAS DE WEAVIATE:")
        print(f"   📦 Total chunks indexados: {current_stats.get('total_chunks', 0):,}")
        print(f"   📄 Documentos únicos: {current_stats.get('total_documents', 0)}")
        
        # Mostrar documentos indexados
        if current_stats.get('documents'):
            print(f"\n📋 DOCUMENTOS EN WEAVIATE:")
            for doc_id, chunk_count in current_stats['documents'].items():
                print(f"   • {doc_id}: {chunk_count:,} chunks")
        
        # Mostrar tipos de chunks
        if current_stats.get('chunk_types'):
            print(f"\n🏷️ TIPOS DE CHUNKS:")
            for chunk_type, count in current_stats['chunk_types'].items():
                print(f"   • {chunk_type}: {count:,}")
        
        # Comparar con archivos originales
        if 'discovered_files' in locals() and discovered_files:
            expected_chunks = sum(f["chunks_count"] for f in discovered_files)
            indexed_chunks = current_stats.get('total_chunks', 0)
            
            print(f"\n🔄 COMPARACIÓN:")
            print(f"   📥 Chunks esperados: {expected_chunks:,}")
            print(f"   📤 Chunks indexados: {indexed_chunks:,}")
            
            if indexed_chunks == expected_chunks:
                print(f"   ✅ ¡Indexación completa al 100%!")
            elif indexed_chunks > 0:
                coverage = (indexed_chunks / expected_chunks) * 100
                print(f"   📊 Cobertura: {coverage:.1f}%")
                if coverage < 100:
                    missing = expected_chunks - indexed_chunks
                    print(f"   ⚠️ Faltan {missing:,} chunks")
            else:
                print(f"   ❌ No hay chunks indexados")
    else:
        print("❌ No se pudieron obtener estadísticas de Weaviate")
else:
    print("⚠️ Sin conexión a Weaviate para validación")

🔍 VALIDANDO INDEXACIÓN...

📊 ESTADÍSTICAS DE WEAVIATE:
   📦 Total chunks indexados: 193,180
   📄 Documentos únicos: 1000

📋 DOCUMENTOS EN WEAVIATE:
   • Mena 2014: 4,131 chunks
   • Mena 2018 CRPC : 4,130 chunks
   • Scott 2024: 1,160 chunks
   • The Postal History Frajola Mayer: 1,158 chunks
   • Catálogo Scott CR 2024: 1,061 chunks
   • Forgeries Articles Raul Hernandez: 1,008 chunks
   • Nordberg Collection Gold: 853 chunks
   • Costa Rica 1881-82 1883 UPU Surcharge Forgeries Fantasies: 800 chunks
   • Pinto Collection: 751 chunks
   • OXCART151: 741 chunks
   • CRF 152: 702 chunks
   • Costa Rica Coat of Arms 1892 Luis Fernando Diaz: 680 chunks
   • Mayer Costa Rica: 656 chunks
   • CRF 75-76: 631 chunks
   • CRF 56: 609 chunks
   • Index2020 Oxcart Subject Final: 603 chunks
   • CRF 49-50: 593 chunks
   • OXCART100: 575 chunks
   • CRF 44-45: 574 chunks
   • filatelista 12 b: 567 chunks
   • filatelista 12 Café Establecimiento de una República: 562 chunks
   • filatelista 12 a: 54

## 6. Pruebas de Búsqueda Semántica

Probar el sistema de búsqueda semántica con consultas filatélicas específicas.

In [38]:
def test_philatelic_searches(client) -> None:
    """
    Ejecutar búsquedas de prueba para validar el sistema.
    """
    if not client:
        print("❌ Sin conexión a Weaviate")
        return
    
    # Consultas de prueba filatélicas
    test_queries = [
        {
            "name": "Búsqueda general de sellos",
            "query": "stamps Scott catalog Costa Rica",
            "filters": None
        },
        {
            "name": "Catálogo Scott específico",
            "query": "Scott catalog numbers",
            "filters": {"catalog_system": "Scott"}
        },
        {
            "name": "Sobrecargas y variedades",
            "query": "overprint surcharge variety error",
            "filters": {"has_varieties": True}
        },
        {
            "name": "Periodo Guanacaste",
            "query": "Guanacaste overprint historical Costa Rica",
            "filters": {"is_guanacaste": True}
        },
        {
            "name": "Especificaciones técnicas",
            "query": "perforation paper printing watermark",
            "filters": {"has_technical_specs": True}
        },
        {
            "name": "Tablas con datos",
            "query": "catalog table prices values",
            "filters": {"chunk_type": "table"}
        }
    ]
    
    print("🔍 EJECUTANDO BÚSQUEDAS DE PRUEBA")
    print("=" * 60)
    
    for i, test in enumerate(test_queries, 1):
        print(f"\n🔎 [{i}/{len(test_queries)}] {test['name']}")
        print(f"   Query: \"{test['query']}\"")
        if test['filters']:
            print(f"   Filtros: {test['filters']}")
        
        try:
            results = search_chunks_semantic(
                client, 
                test["query"], 
                "Oxcart", 
                limit=3,
                filters=test["filters"]
            )
            
            print(f"   📊 Resultados: {len(results)}")
            
            for j, result in enumerate(results, 1):
                print(f"\n      🏷️ #{j} (Score: {result['score']:.3f})")
                print(f"         📄 Documento: {result['doc_id']}")
                print(f"         📋 Tipo: {result['chunk_type']}")
                print(f"         📄 Página: {result['page_number']}")
                
                # Mostrar metadatos relevantes
                if result.get('catalog_systems'):
                    print(f"         📖 Catálogos: {result['catalog_systems']}")
                if result.get('scott_numbers'):
                    print(f"         🔢 Scott: {result['scott_numbers']}")
                if result.get('years'):
                    print(f"         📅 Años: {result['years']}")
                if result.get('colors'):
                    print(f"         🎨 Colores: {result['colors']}")
                if result.get('variety_classes'):
                    print(f"         🔀 Variedades: {result['variety_classes']}")
                
                # Texto truncado
                text = result.get('text', '')
                if len(text) > 200:
                    text = text[:200] + "..."
                print(f"         📝 Texto: {text}")
            
            if not results:
                print(f"   ⚠️ No se encontraron resultados")
            
        except Exception as e:
            print(f"   ❌ Error en búsqueda: {e}")
        
        print("   " + "-" * 50)

# # Ejecutar pruebas de búsqueda
# if client:
#     test_philatelic_searches(client)
# else:
#     print("⚠️ No se pueden ejecutar búsquedas sin conexión a Weaviate")

In [39]:
results = search_chunks_semantic(
                client, 
                "Costa Rica 1907 2 colones stamp with original gum. Scott 68 issue of 1907", 
                "Oxcart", 
                limit=100,
                filters=[],
                mode = "hybrid",
                alpha= 0.45
                
            )
            
print(f"   📊 Resultados: {len(results)}")

for j, result in enumerate(results, 1):
    print(f"\n      🏷️ #{j} (Score: {result['score']:.3f})")
    print(f"         📄 Documento: {result['doc_id']}")
    print(f"         📋 Tipo: {result['chunk_type']}")
    print(f"         📄 Página: {result['page_number']}")
    
    # Mostrar metadatos relevantes
    if result.get('catalog_systems'):
        print(f"         📖 Catálogos: {result['catalog_systems']}")
    if result.get('scott_numbers'):
        print(f"         🔢 Scott: {result['scott_numbers']}")
    if result.get('years'):
        print(f"         📅 Años: {result['years']}")
    if result.get('colors'):
        print(f"         🎨 Colores: {result['colors']}")
    if result.get('variety_classes'):
        print(f"         🔀 Variedades: {result['variety_classes']}")
    
    # Texto truncado
    text = result.get('text', '')
    # if len(text) > 200:
    #     text = text[:200] + "..."
    print(f"         📝 Texto: {text}")
    print("**********************************************************************************************************")

   📊 Resultados: 100

      🏷️ #1 (Score: 0.550)
         📄 Documento: OXCART116
         📋 Tipo: text
         📄 Página: 25
         📖 Catálogos: ['Scott']
         🔢 Scott: ['2, 3, 4', '32-34, 35–44', '4, 1', '64, 65, 66', '68', '143-146', '32-34', '35–44', '68, 143-146']
         📅 Años: [1907]
         🎨 Colores: ['red']
         📝 Texto: Got any ideas?\n\nSuggestions for the improvement of the OXCART Postal Sales are always welcome!\nCondición: centrado fine.\n\n![Figure](figures/OXCART116_page_025_figure_000.png)\nCondición: centrado good.\n\n192 193 194.\n\n![Figure](figures/OXCART116_page_025_figure_004.png)\nCondición: centrado good.\n\n![Figure](figures/OXCART116_page_025_figure_006.png)\nCondición: centrado good.\n\n196 197. 198. 199. 200.\n\n![Figure](figures/OXCART116_page_025_figure_012.png)\nCondición: centrado good.\n\n![Figure](figures/OXCART116_page_025_figure_013.png)\nCondición: centrado good.\n\n![Figure](figures/OXCART116_page_025_figure_014.png)\nCondición: centr

## 7. Interfaz Gradio para RAG

Interfaz web interactiva para búsquedas semánticas y consultas RAG.

In [40]:
# try:
#     import gradio as gr
#     import openai
#     gradio_available = True
#     print("✅ Gradio disponible")
# except ImportError:
#     print("⚠️ Gradio no está instalado")
#     print("💡 Para instalar: pip install gradio")
#     gradio_available = False

# # Configurar OpenAI para RAG
# if OPENAI_API_KEY:
#     openai.api_key = OPENAI_API_KEY
#     openai_available = True
# else:
#     openai_available = False
#     print("⚠️ OpenAI API key no configurada para RAG")

In [41]:
# import os
# from typing import Any, Dict, List, Tuple

# def search_and_answer(
#     query: str,
#     rag_system: Dict[str, Any],
#     use_filters: bool = False,
#     catalog_system: str = "",
#     chunk_type: str = "",
#     has_varieties: bool = False,
#     max_results: int = 10,
# ) -> Tuple[str, List[Dict[str, Any]], Dict[str, Any]]:
#     """
#     Búsqueda semántica + RAG (OpenAI >= 1.0, modelo gpt-4o-mini).
#     Devuelve: (respuesta_rag, resultados(lista de dicts), metadatos(dict))
#     """
#     # Validación de conexión
#     if not rag_system or not rag_system.get("client"):
#         meta = {"query": query, "total_results": 0, "max_results": max_results, "filters_used": {}, "context_length": 0}
#         return "❌ Error: Sin conexión a Weaviate", [], meta

#     client_wv = rag_system["client"]
#     collection_name = rag_system.get("collection_name", "Oxcart")

#     # Construir filtros
#     filt = None
#     if use_filters:
#         filt = {}
#         if catalog_system:
#             filt["catalog_system"] = catalog_system
#         if chunk_type:
#             filt["chunk_type"] = chunk_type
#         if has_varieties:
#             filt["has_varieties"] = True

#     # Búsqueda semántica (usa tu función ya definida)
#     results = search_chunks_semantic(
#         client=client_wv,
#         query=query,
#         collection_name=collection_name,
#         limit=int(max_results),
#         filters=filt,
#         mode = "hybrid",
#         alpha= 0.35
#     )

#     # Preparar contexto para RAG (top 3)
#     top = results[:3]
#     context = "\n\n".join(
#         f"Documento {r.get('doc_id', 'N/A')} (Página {r.get('page_number', '¿?')}): {r.get('text','')}"
#         for r in top
#     )
#     context_len = len(context)

#     # Generar respuesta RAG (OpenAI >= 1.0.0)
#     rag_answer = "⚠️ No se encontraron resultados para generar respuesta"
#     openai_key = os.getenv("OPENAI_API_KEY")
#     if not results:
#         rag_answer = "⚠️ No se encontraron resultados para generar respuesta"
#     elif not openai_key:
#         rag_answer = "⚠️ RAG no disponible: OpenAI API key no configurada"
#     else:
#         try:
#             from openai import OpenAI
#             oa_client = OpenAI(api_key=openai_key)

#             system_prompt = (
#                 "You are an expert in costa rica philately (stamps, covers, etc). "
#                 "Only answer based with the information provided. If there is not enough info for answer please, "
#                 "answer with: 'I dont have information'. You must include any references about philatelic like scott catalogue references, dates, etc."
#             )

#             model = os.getenv("RAG_MODEL", "gpt-4o-mini")
#             resp = oa_client.chat.completions.create(
#                 model=model,
#                 messages=[
#                     {"role": "system", "content": system_prompt},
#                     {"role": "user", "content": f"Here is the information for your answers:\n{context}\n\nAnswer this only with the information provided: {query}"}
#                 ],
#                 temperature=0.3,
#                 max_tokens=1000,
#             )

#             rag_text = resp.choices[0].message.content if resp.choices else ""
#             if not rag_text:
#                 rag_text = "No se obtuvo texto de respuesta del modelo."

#             rag_answer = (
#                 "🤖 **Respuesta RAG:**\n\n"
#                 + rag_text
#                 + f"\n\n📊 *Basado en {len(results)} resultados de búsqueda*"
#             )
#         except Exception as e:
#             rag_answer = f"❌ Error generando respuesta RAG: {e}"

#     metadata = {
#         "query": query,
#         "total_results": len(results),
#         "max_results": int(max_results),
#         "filters_used": filt or {},
#         "context_length": context_len,
#     }
#     return rag_answer, results, metadata


In [42]:
# def get_collection_info() -> str:
#     """
#     Obtener información de la colección para mostrar en la interfaz.
#     """
#     if not client:
#         return "❌ Sin conexión a Weaviate"
    
#     try:
#         stats = get_collection_stats(client, "Oxcart")
#         if stats:
#             info = f"📊 **Estadísticas de la Colección Oxcart:**\n\n"
#             info += f"📦 **Total chunks:** {stats['total_chunks']:,}\n"
#             info += f"📄 **Documentos:** {stats['total_documents']}\n\n"
            
#             if stats.get('documents'):
#                 info += "**Documentos indexados:**\n"
#                 for doc_id, count in stats['documents'].items():
#                     info += f"• {doc_id}: {count:,} chunks\n"
            
#             return info
#         else:
#             return "❌ No se pudieron obtener estadísticas"
#     except Exception as e:
#         return f"❌ Error: {e}"

# print("✅ Funciones RAG definidas")

In [43]:
# stats = get_collection_stats(client, "Oxcart")
# stats['total_documents']
# stats['total_chunks']

In [44]:
# # Estructura que usan tus funciones de búsqueda/respuesta
# rag_system = {
#     "success": True,
#     "client": client,                    # para que search_and_answer pueda consultar
#     "collection_name": COLLECTION_NAME,  # nombre de la colección
#     "weaviate_url": WEAVIATE_URL,        # info para la UI
#     "total_documents": stats['total_documents'],       # para mostrar estado
#     "total_chunks": stats['total_chunks'],        # opcional en la UI
#     # puedes añadir más campos que tu search_and_answer necesite
# }

In [45]:
# import os
# import gradio as gr
# from typing import Dict, Any
# import threading
# import time

# def create_gradio_interface(rag_system: Dict[str, Any]) -> gr.Blocks:
#     """
#     Crea la interfaz Gradio para consultas RAG.
#     """

#     def gradio_search_and_answer(query, use_filters, catalog_system, chunk_type, has_varieties, max_results):
#         """
#         Wrapper para Gradio: llama a search_and_answer y formatea salidas.
#         """
#         if not rag_system:
#             return "❌ Sistema RAG no está configurado", "No hay resultados", "No hay metadatos"

#         # Llamada a tu función (se asume definida en tu entorno)
#         answer, results, metadata = search_and_answer(
#             query=query,
#             rag_system=rag_system,
#             use_filters=use_filters,
#             catalog_system=catalog_system,
#             chunk_type=chunk_type,
#             has_varieties=has_varieties,
#             max_results=int(max_results),
#         )

#         # --- Formatear resultados de búsqueda ---
#         lines = []
#         if results:
#             for i, r in enumerate(results):
#                 doc_id = r.get("doc_id") or r.get("document_id", "N/A")
#                 chunk_type_val = r.get("chunk_type", "N/A")
#                 page_number = r.get("page_number", "N/A")
#                 catalogs = r.get("catalog_systems") or []
#                 scotts = r.get("scott_numbers") or []
#                 years = r.get("years") or []

#                 # Vista previa: usa content_preview si existe; si no, toma 'text'
#                 preview = r.get("content_preview")
#                 if not preview:
#                     text = r.get("text", "")
#                     preview = (text[:300] + "...") if len(text) > 300 else text

#                 block = []
#                 block.append(f"**Resultado {i+1}**")
#                 block.append(f"• Documento: {doc_id}")
#                 block.append(f"• Tipo: {chunk_type_val} | Página: {page_number}")
#                 if catalogs:
#                     block.append(f"• Catálogos: {', '.join(catalogs)}")
#                 if scotts:
#                     block.append(f"• Scott: {', '.join(scotts)}")
#                 if years:
#                     block.append(f"• Años: {', '.join(str(y) for y in years)}")
#                 block.append(f"• Vista previa: {preview}")
#                 block.append("-" * 50)
#                 lines.append("\n".join(block))
#             search_output = "\n".join(lines)
#         else:
#             search_output = "No se encontraron resultados"

#         # --- Formatear metadatos ---
#         metadata = metadata or {}
#         metadata_output = (
#             "**Metadatos de la consulta:**\n"
#             f"• Consulta: {metadata.get('query', 'N/A')}\n"
#             f"• Resultados encontrados: {metadata.get('total_results', 0)}\n"
#             f"• Máximo solicitado: {metadata.get('max_results', 'N/A')}\n"
#             f"• Filtros usados: {metadata.get('filters_used', {})}\n"
#             f"• Longitud del contexto: {metadata.get('context_length', 'N/A')} caracteres\n"
#         )

#         return answer, search_output, metadata_output

#     # Valores informativos del sistema
#     collection_name = rag_system.get("collection_name", "Oxcart")
#     total_docs = rag_system.get("total_documents", 0)
#     weaviate_url = rag_system.get("weaviate_url") or os.getenv("WEAVIATE_URL", "http://localhost:8080")

#     # --- UI ---
#     with gr.Blocks(title="OXCART RAG - Consultas Filatélicas") as interface:
#         gr.Markdown(
#             "# 🔍 OXCART RAG - Sistema de Consultas Filatélicas\n\n"
#             "Realiza consultas inteligentes sobre tu colección de documentos filatélicos "
#             "usando búsqueda semántica y respuestas generadas por IA."
#         )

#         with gr.Row():
#             with gr.Column(scale=2):
#                 # Input principal
#                 query_input = gr.Textbox(
#                     label="💭 Tu consulta filatélica",
#                     placeholder="Ej: ¿Qué sellos de España de 1950 están catalogados como Scott?",
#                     lines=2,
#                 )

#                 # Botón de búsqueda
#                 search_btn = gr.Button("🔍 Buscar y Responder", variant="primary")

#                 # Consultas de ejemplo
#                 gr.Markdown("**💡 Consultas de ejemplo:**")
#                 example_queries = [
#                     "¿Qué sellos conmemorativos de España están en la colección?",
#                     "Muéstrame información sobre sellos con errores de perforación",
#                     "¿Cuáles son los sellos más valiosos según el catálogo Michel?",
#                     "Información sobre sellos de México de la década de 1960",
#                     "¿Qué variedades filatélicas están documentadas?",
#                 ]
#                 # Botones que rellenan el textbox
#                 for example in example_queries:
#                     gr.Button(example, variant="secondary").click(
#                         fn=(lambda ex=example: ex),
#                         inputs=None,
#                         outputs=query_input,
#                     )

#             with gr.Column(scale=1):
#                 # Filtros avanzados
#                 gr.Markdown("**🎯 Filtros Avanzados**")

#                 use_filters = gr.Checkbox(label="Usar filtros específicos", value=False)

#                 catalog_system = gr.Dropdown(
#                     choices=["", "Scott", "Michel", "Yvert", "Stanley Gibbons", "Edifil"],
#                     label="Sistema de catálogo",
#                     value="",
#                 )

#                 chunk_type = gr.Dropdown(
#                     choices=["", "text", "table", "figure", "title", "header"],
#                     label="Tipo de contenido",
#                     value="",
#                 )

#                 has_varieties = gr.Checkbox(label="Solo documentos con variedades", value=False)

#                 max_results = gr.Slider(
#                     minimum=1,
#                     maximum=100,
#                     value=5,
#                     step=1,
#                     label="Máximo resultados",
#                 )

#         # Outputs
#         with gr.Row():
#             with gr.Column():
#                 gr.Markdown("## 🤖 Respuesta IA")
#                 answer_output = gr.Textbox(label="Respuesta generada", lines=8, interactive=False)

#         with gr.Row():
#             with gr.Column():
#                 gr.Markdown("## 📄 Documentos Encontrados")
#                 search_output = gr.Textbox(label="Resultados de búsqueda", lines=12, interactive=False)

#             with gr.Column():
#                 gr.Markdown("## 📊 Metadatos")
#                 metadata_output = gr.Textbox(label="Información de la consulta", lines=10, interactive=False)

#         # Eventos
#         search_btn.click(
#             fn=gradio_search_and_answer,
#             inputs=[query_input, use_filters, catalog_system, chunk_type, has_varieties, max_results],
#             outputs=[answer_output, search_output, metadata_output],
#         )

#         query_input.submit(
#             fn=gradio_search_and_answer,
#             inputs=[query_input, use_filters, catalog_system, chunk_type, has_varieties, max_results],
#             outputs=[answer_output, search_output, metadata_output],
#         )

#         # Información del sistema
#         gr.Markdown(
#             "---\n"
#             f"**📊 Estado del Sistema:**\n"
#             f"• Colección: {collection_name}\n"
#             f"• Documentos indexados: {total_docs:,}\n"
#             f"• Weaviate URL: {weaviate_url}\n"
#             "• Estado: ✅ Operativo\n"
#         )

#     return interface


# # ---- Lanzador robusto con manejo de errores de túnel público ----
# if rag_system and rag_system.get("success", False):
#     print("\n" + "=" * 60)
#     print("🚀 LANZANDO INTERFAZ GRADIO (CON MANEJO DE ERRORES)")
#     print("=" * 60)

#     gradio_app = create_gradio_interface(rag_system)

#     GRADIO_PORT = int(os.getenv("GRADIO_PORT", 7860))
#     GRADIO_SHARE = os.getenv("GRADIO_SHARE", "false").lower() == "true"  # Por defecto False por problemas de conectividad

#     print(f"⚙️ Puerto: {GRADIO_PORT}")
#     print(f"🌍 URL Pública: {'⚠️ Intentando...' if GRADIO_SHARE else '❌ Deshabilitada (más seguro)'}")
    
#     try:
#         print("🔄 Iniciando servidor Gradio...")
        
#         # Intentar con túnel público primero si está habilitado
#         if GRADIO_SHARE:
#             print("⏳ Intentando crear túnel público...")
#             try:
#                 demo = gradio_app.launch(
#                     server_port=GRADIO_PORT,
#                     share=True,
#                     inbrowser=False,
#                     show_error=True,
#                     prevent_thread_lock=False,
#                     quiet=False
#                 )
                
#                 print("\n🎉 ¡ÉXITO! Túnel público creado")
#                 print(f"🌐 URLs DISPONIBLES:")
#                 print(f"   📱 Local: http://localhost:{GRADIO_PORT}")
                
#                 if hasattr(demo, 'share_url') and demo.share_url:
#                     print(f"   🌍 Pública: {demo.share_url}")
#                     print(f"\n🔗 **URL PÚBLICA:** {demo.share_url}")
#                 else:
#                     print(f"   🌍 Pública: Revisa la salida de Gradio arriba ☝️")
                
#             except Exception as share_error:
#                 print(f"⚠️ Error creando túnel público: {share_error}")
#                 print("🔄 Cambiando a modo local solamente...")
                
#                 # Fallback: solo local
#                 demo = gradio_app.launch(
#                     server_port=GRADIO_PORT,
#                     share=False,
#                     inbrowser=True,
#                     show_error=True,
#                     prevent_thread_lock=False
#                 )
                
#                 print(f"\n✅ SERVIDOR LOCAL OPERATIVO:")
#                 print(f"   📱 URL Local: http://localhost:{GRADIO_PORT}")
#                 print(f"   ⚠️ URL Pública: No disponible (error en túnel)")
                
#         else:
#             # Solo modo local
#             demo = gradio_app.launch(
#                 server_port=GRADIO_PORT,
#                 share=False,
#                 inbrowser=True,
#                 show_error=True,
#                 prevent_thread_lock=False
#             )
            
#             print(f"\n✅ SERVIDOR LOCAL OPERATIVO:")
#             print(f"   📱 URL Local: http://localhost:{GRADIO_PORT}")
#             print(f"   💡 Para URL pública, cambia GRADIO_SHARE=true en .env")
        
#         print(f"\n📋 INSTRUCCIONES:")
#         print(f"   • La interfaz está operativa y lista para consultas")
#         print(f"   • Para detenerla: gr.close_all()")
#         print(f"   • Comparte la URL local en tu red si necesitas acceso remoto")
        
#         print(f"\n{'='*60}")
#         print(f"🎯 INTERFAZ RAG LISTA - ¡Comienza a hacer consultas!")
#         print(f"{'='*60}")
        
#     except Exception as e:
#         print(f"❌ Error crítico lanzando Gradio: {e}")
#         print("\n🔧 SOLUCIONES SUGERIDAS:")
#         print("   1. Ejecuta: gr.close_all()")
#         print("   2. Cambia el puerto: GRADIO_PORT=7861 en .env")
#         print("   3. Verifica que no hay otros servicios en el puerto")
#         print("   4. Reinicia el notebook")
        
# else:
#     print("\n⚠️  No se puede crear la interfaz Gradio:")
#     if not rag_system:
#         print("   • Sistema RAG no está configurado")
#     else:
#         print(f"   • Error en RAG: {rag_system.get('error', 'Error desconocido')}")
#     print("\n🔧 Para solucionar:")
#     print("   1. Verifica que Weaviate esté corriendo")
#     print("   2. Configura OPENAI_API_KEY en .env") 
#     print("   3. Ejecuta la indexación de documentos")
#     print("   4. Reinicia este notebook")

In [46]:
# # Cerrar instancias anteriores de Gradio si existen
# import gradio as gr
# gr.close_all()
# print("🔄 Cerrando instancias anteriores de Gradio")